### Užduotis

Pirma užduotis reikalaus realizuoti:

- [X] efektyvią duomenų nuskaitymo programą su pasirinku egzistuojančiu iš anksto apmokytu (angl. pre-trained) vaizdų klasifikavimo modeliu,
- [X] paskaičiuoti tikslumo, precizijos, atkūrimo ir F1 statistikas pasirinktiems 1000 paveikslėlių iš OpenImages,
- [X] realizuoti slenkstinės reikšmės (angl. threshold) keitimą, įgalinant klasifikuoti vaizdus kiekvienai užduotai klasei keičiant . Statistikos turi persiskaičiuoti po slenkstinės reikšmės pakeitimo.

In [10]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [11]:
from pathlib import Path
from typing import Callable
from torch import Tensor
from torch.utils.data import Dataset
from PIL import Image
from importlib import reload
from copy import deepcopy
from lab1.mydataset import MyDataSet
import lab1

In [12]:
from lab1.device import device
reload(lab1.device)
device

device(type='mps')

In [13]:
from lab1.download import download_if_not_found
reload(lab1.download)
download_if_not_found()

Downloading: Container


               0           1
0      /m/011k07    Tortoise
1    /m/011q46kg   Container
2      /m/012074      Magpie
3      /m/0120dh  Sea turtle
4      /m/01226z    Football
..           ...         ...
596     /m/0qmmr  Wheelchair
597   /m/0wdt60w  Rugby ball
598      /m/0xfy   Armadillo
599     /m/0xzly     Maracas
600     /m/0zvk5      Helmet

[601 rows x 2 columns]


In [ ]:
from torchvision.models import resnet101, ResNet101_Weights

weights: ResNet101_Weights = ResNet101_Weights.DEFAULT

model = resnet101(weights=weights, progress=True).to(device).eval()

catmap = dict([(i, c) for c, i in enumerate(weights.meta['categories'])])

{'tench': 0,
 'goldfish': 1,
 'great white shark': 2,
 'tiger shark': 3,
 'hammerhead': 4,
 'electric ray': 5,
 'stingray': 6,
 'cock': 7,
 'hen': 8,
 'ostrich': 9,
 'brambling': 10,
 'goldfinch': 11,
 'house finch': 12,
 'junco': 13,
 'indigo bunting': 14,
 'robin': 15,
 'bulbul': 16,
 'jay': 17,
 'magpie': 18,
 'chickadee': 19,
 'water ouzel': 20,
 'kite': 21,
 'bald eagle': 22,
 'vulture': 23,
 'great grey owl': 24,
 'European fire salamander': 25,
 'common newt': 26,
 'eft': 27,
 'spotted salamander': 28,
 'axolotl': 29,
 'bullfrog': 30,
 'tree frog': 31,
 'tailed frog': 32,
 'loggerhead': 33,
 'leatherback turtle': 34,
 'mud turtle': 35,
 'terrapin': 36,
 'box turtle': 37,
 'banded gecko': 38,
 'common iguana': 39,
 'American chameleon': 40,
 'whiptail': 41,
 'agama': 42,
 'frilled lizard': 43,
 'alligator lizard': 44,
 'Gila monster': 45,
 'green lizard': 46,
 'African chameleon': 47,
 'Komodo dragon': 48,
 'African crocodile': 49,
 'American alligator': 50,
 'triceratops': 51,
 

In [ ]:
from lab1.trans import img_trans_1

img_trans_2 = weights.transforms()
dataset = MyDataSet("data", preprocess_fn = img_trans_1, label_map = catmap)

In [ ]:
from torch.utils.data import DataLoader

data_loader = DataLoader(dataset, batch_size = 2**4, num_workers=0)

In [ ]:
from functools import reduce
import numpy as np
import torch
from torch import cat
from torch.utils.data import Dataset
from torchvision.transforms import Compose, Resize, ToTensor, Normalize

predictions = torch.empty((0, 1000))
truths_ = torch.tensor([], dtype=int)

for images, ts in data_loader:
    truths_  = torch.cat((truths_, ts))
    # Can be though as list of ŷ
    #     ŷ_i - confidence that input image belongs to i-th class
    #              "squishes" outputs to a range of [0, 1] 👇
    from_model = model(images.to(device)).detach().cpu().sigmoid()
    predictions = cat((predictions, from_model))

# Initializes list of vectors:
#   truths - 
#   y - ground truth vector, with only one (i-th element), representing a class set to true
truths = torch.zeros_like(predictions, dtype=torch.bool)
for index, label in enumerate(truths_):
    truths[index,label.item()] = True

truths, predictions

(tensor([[False,  True, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False,  True, False,  ..., False, False, False]]),
 tensor([[0.5425, 0.9935, 0.6441,  ..., 0.3833, 0.4565, 0.5783],
         [0.6684, 0.3726, 0.4642,  ..., 0.8267, 0.4377, 0.2817],
         [0.4210, 0.3795, 0.4725,  ..., 0.3582, 0.3331, 0.8560],
         ...,
         [0.5852, 0.4148, 0.5887,  ..., 0.6366, 0.5322, 0.5069],
         [0.6958, 0.3480, 0.3711,  ..., 0.4335, 0.9807, 0.4671],
         [0.5257, 0.9984, 0.8835,  ..., 0.3257, 0.4254, 0.4345]]))

In [ ]:
from lab1.stat import table, stats
def test():
    tp, tn, fp, fn = table(truths, predictions)
    return tp + tn + fp + fn == truths_.numel()

table(truths, predictions), test()


((tensor(315), tensor(830), tensor(220), tensor(19)), tensor(True))

In [ ]:
from lab1.download import classes

for label in classes:
    print(f"---------------------")
    print(f"Class: {label}")
    acc, re, pre, f1 = stats(table(truths, predictions, labels=catmap[label.lower()], threshold = 0.8))
    print(f" . Accuracy:  {acc:.3}\n . Recall:    {re:.3}\n . Precision: {pre:.3}\n . F1:        {f1:.3}\n")

---------------------
Class: Banana
 . Accuracy:  0.82
 . Recall:    0.96
 . Precision: 0.588
 . F1:        0.73

---------------------
Class: Snail
 . Accuracy:  0.78
 . Recall:    0.974
 . Precision: 0.536
 . F1:        0.692

---------------------
Class: Vase
 . Accuracy:  0.783
 . Recall:    0.914
 . Precision: 0.542
 . F1:        0.681

---------------------
Class: Goldfish
 . Accuracy:  0.827
 . Recall:    0.943
 . Precision: 0.589
 . F1:        0.725

